Requirements

In [ ]:
!pip install inltk

from __future__ import unicode_literals, print_function, division
from io import open
import inltk
from inltk.inltk import tokenize
from inltk.inltk import setup
import unicodedata
import string
import re
import pandas as pd
import random
import os
import csv
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from nltk.translate.bleu_score import sentence_bleu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from google.colab import drive
drive.mount("/content/drive")

     |████████████████████████████████| 81kB 2.9MB/s 
     |████████████████████████████████| 1.2MB 8.2MB/s 
     |████████████████████████████████| 235kB 40.6MB/s 
     |████████████████████████████████| 1.1MB 39.5MB/s 
     |████████████████████████████████| 256kB 40.7MB/s 
     |████████████████████████████████| 153kB 38.4MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-cp36-none-any.whl size=26309 sha256=ce050c6eb177bf5f94eddcfc833bd28b9e4ec2f456cc51f4fde74b0bbf4d930e
  Stored in directory: /root/.cache/pip/wheels/2d/04/41/8e1836e79581989c22eebac3f4e70aaac9af07b0908da173be
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=2237041b04f9a8d89730f0d5f35120ae45a7affd91a268251b027540d2cf81ea
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built typing idna-ssl
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fas

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 2000

#initialize Lang Class
class Lang:
   def __init__(self):
       #initialize containers to hold the words and corresponding index
       self.word2index = {}
       self.word2count = {}
       self.index2word = {0: "SOS", 1: "EOS"}
       self.n_words = 2  # Count SOS and EOS

#split a sentence into words and add it to the container
   def addSentence(self, sentence):
       for word in sentence.split(' '):
           self.addWord(word)

#If the word is not in the container, the word will be added to it, 
#else, update the word counter
   def addWord(self, word):
       if word not in self.word2index:
           self.word2index[word] = self.n_words
           self.word2count[word] = 1
           self.index2word[self.n_words] = word
           self.n_words += 1
       else:
           self.word2count[word] += 1

In [ ]:
def process_data():

  sf = open('/content/drive/My Drive/data/bible-uedin.hi-ne.hi' , "r")
  tf = open('/content/drive/My Drive/data/bible-uedin.hi-ne.ne' , "r")

  source = Lang()
  target = Lang()
  pairs = []
  count = 0
  count2 = 0
  for sent in sf:
    sent = sent.strip()
    source.addSentence(sent)
    pairs.append(sent)
    count = count+1

  # print(count)

  for sent in tf:
    sent = sent.strip()
    target.addSentence(sent.strip())
    pairs.append(sent)
    count2 = count2 + 1

  # print(count2)

  pairs_new = []

  for i in range(count):
    full = [pairs[i],pairs[i+count]]
    pairs_new.append(full)



  return source, target, pairs_new

In [ ]:
# source, target, pairs = process_data()   JUST A CHECK TO SEE IF THE SIZE OF BOTH DATASET IS SAME

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


Model

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


Training the model

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        # plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

    #     if iter % plot_every == 0:
    #         plot_loss_avg = plot_loss_total / plot_every
    #         plot_losses.append(plot_loss_avg)
    #         plot_loss_total = 0

    # showPlot(plot_losses)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]


def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(val_pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')        

In [ ]:
input_lang, output_lang, pairs = process_data()
pairs = filterPairs(pairs)

print(random.choice(pairs))

print(len(pairs))
x = len(pairs)

trainSplit = int(x*0.7)
valSplit = trainSplit + int(x*0.2)

train_pairs = pairs[:trainSplit]
val_pairs = pairs[trainSplit:valSplit]
test_pairs = pairs[valSplit:]

print(len(train_pairs))
print(len(val_pairs))
print(len(test_pairs))

['मैं तुम को इस में से निकालकर परदेशियों के हाथ में कर दूंगा, और तुम को दण्ड दिलाऊंगा।', 'परमेश्वरले यो पनि भन्नुभयो, “म तिमीहरूलाई यो शहरबाट बाहिर निकाल्नेछु अनि तिमीहरूलाई अपरिचितहरूको हातमा सुम्पिदिनेछु। म तिमीहरूलाई दण्ड दिनेछु!']
30486
21340
6097
3049


In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


In [ ]:
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 30000, print_every=1000)

2m 47s (- 80m 56s) (1000 3%) 6.5509
5m 32s (- 77m 41s) (2000 6%) 6.4346
8m 16s (- 74m 27s) (3000 10%) 6.2416
10m 58s (- 71m 22s) (4000 13%) 6.1660
13m 37s (- 68m 7s) (5000 16%) 6.2259
16m 16s (- 65m 5s) (6000 20%) 6.1298
19m 0s (- 62m 26s) (7000 23%) 6.1571
21m 47s (- 59m 54s) (8000 26%) 6.1937
24m 32s (- 57m 16s) (9000 30%) 6.1979
27m 17s (- 54m 34s) (10000 33%) 6.0655
30m 0s (- 51m 50s) (11000 36%) 6.0530
32m 44s (- 49m 6s) (12000 40%) 6.0149
35m 26s (- 46m 21s) (13000 43%) 6.0222
38m 12s (- 43m 39s) (14000 46%) 5.9743
40m 57s (- 40m 57s) (15000 50%) 6.1086
43m 46s (- 38m 17s) (16000 53%) 6.1033
46m 36s (- 35m 38s) (17000 56%) 6.1417
49m 24s (- 32m 56s) (18000 60%) 5.9593
52m 20s (- 30m 18s) (19000 63%) 6.0595
55m 12s (- 27m 36s) (20000 66%) 5.9755
58m 5s (- 24m 53s) (21000 70%) 6.0166
61m 2s (- 22m 11s) (22000 73%) 6.0236
63m 59s (- 19m 28s) (23000 76%) 6.0507
66m 51s (- 16m 42s) (24000 80%) 5.9018
69m 49s (- 13m 57s) (25000 83%) 5.9767
72m 45s (- 11m 11s) (26000 86%) 5.8544
75m 39s

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

> और उन्हों ने पत्थर को कब्र पर से लुढ़का हुआ पाया।
= चिहानको प्रवेशद्वार बन्द गर्नलाई एउटा ठूलो ढुङ्गा राखिएको थियो। तर तिनीहरूले त्यो ढुङ्गा चिहानदेखि कतै गुड्याइ पठाएको पाए।
< त्यसपछि मानिसहरू मानिसहरू अनि अनि अनि अनि मानिसहरू अनि अनि अनि <EOS>

> पर मैं तुम से कहता हूं, कि न्याय के दिन तेरी दशा से सदोम के देश की दशा अधिक सहने योग्य होगी।
= तर तिमीलाई भन्छु कि न्यायको दिनमा सदोमको भन्दा तिम्रो हालत अरू नराम्रो हुनेछ।”
< म तिम्रो म म म अनि म म अनि म अनि म अनि म अनि म अनि म अनि म <EOS>

> और उस ने उन से कहा, कि जब मैं ने तुम्हें बटुए, और झोली, और जूते बिना भेजा था, तो क्या तुम को किसी वस्तु की घटी हुई थी? उन्हों ने कहा; किसी वस्तु की नहीं।
= तब येशूले प्रेरितहरूलाई भन्नुभयो, “जब मैले तिमीहरूले मानिसहरूमा प्रचार गर्न, विना पैसा, झोला अनि जुत्ताहरू विना पठाएँ, के तिमीहरूलाई केही अभाव भयो?” प्रेरितहरूले भने, “भएन।”
< तर ती मानिसहरू भने, भने, मानिसहरू अनि म अनि अनि म अनि म अनि म अनि म अनि अनि म अनि अनि म अनि म अनि <EOS>

> और जब वहां पहुंचे तो वे उस अटारी पर गए, जहां पतरस और यूहन्ना और या

In [ ]:
class Seq2Seq(nn.Module):
   def __init__(self, encoder, decoder, device, MAX_LENGTH=MAX_LENGTH):
       super().__init__()
      
#initialize the encoder and decoder
       self.encoder = encoder
       self.decoder = decoder
       self.device = device
     
   def forward(self, source, target, teacher_forcing_ratio=0.5):

       input_length = source.size(0) #get the input length (number of words in sentence)
       batch_size = target.shape[1] 
       target_length = target.shape[0]
       vocab_size = self.decoder.output_dim
      
#initialize a variable to hold the predicted outputs
       outputs = torch.zeros(target_length, batch_size, vocab_size).to(self.device)

#encode every word in a sentence
       for i in range(input_length):
           encoder_output, encoder_hidden = self.encoder(source[i])

#use the encoder’s hidden layer as the decoder hidden
       decoder_hidden = encoder_hidden.to(device)
  
#add a token before the first predicted word
       decoder_input = torch.tensor([SOS_token], device=device)  # SOS

#topk is used to get the top K value over a list
#predict the output word from the current target word. If we enable the teaching force,  then the #next decoder input is the next word, else, use the decoder output highest value. 

       for t in range(target_length):   
           decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
           outputs[t] = decoder_output
           teacher_force = random.random() < teacher_forcing_ratio
           topv, topi = decoder_output.topk(1)
           input = (target[t] if teacher_force else topi)
           if(teacher_force == False and input.item() == EOS_token):
               break

       return outputs

In [ ]:
model = Seq2Seq(encoder1, attn_decoder1, device).to(device)

In [ ]:
for name, params in model.named_children():
  print(name)

encoder
decoder


In [ ]:
for param in model.parameters():    
    param.requires_grad = False

trained_encoder = list(model.children())[0]
trained_decoder = list(model.children())[1] 

In [ ]:
encoder2 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder2 = trained_decoder

trainIters(encoder2, attn_decoder2, 15000, print_every=1000)

1m 40s (- 23m 33s) (1000 6%) 6.3637
3m 16s (- 21m 20s) (2000 13%) 6.0927
4m 51s (- 19m 27s) (3000 20%) 6.0425
6m 28s (- 17m 48s) (4000 26%) 6.1265
8m 6s (- 16m 13s) (5000 33%) 6.0086
9m 45s (- 14m 37s) (6000 40%) 6.0939
11m 23s (- 13m 1s) (7000 46%) 6.0783
13m 5s (- 11m 26s) (8000 53%) 6.0864
14m 43s (- 9m 49s) (9000 60%) 6.0961
16m 23s (- 8m 11s) (10000 66%) 6.1685
18m 3s (- 6m 33s) (11000 73%) 6.1157
19m 42s (- 4m 55s) (12000 80%) 6.0090
21m 21s (- 3m 17s) (13000 86%) 5.9824
23m 2s (- 1m 38s) (14000 93%) 6.0688
24m 45s (- 0m 0s) (15000 100%) 6.1751


In [ ]:
new_model = Seq2Seq(encoder2, attn_decoder2, device).to(device)

for param in new_model.parameters():    
    param.requires_grad = True

trained_encoder = list(new_model.children())[0]
trained_decoder = list(new_model.children())[1] 

trainIters(trained_encoder, trained_decoder, 15000, print_every=1000)

2m 58s (- 41m 44s) (1000 6%) 5.9711
5m 51s (- 38m 2s) (2000 13%) 5.8885
8m 51s (- 35m 25s) (3000 20%) 5.7722
11m 49s (- 32m 31s) (4000 26%) 5.9127
14m 47s (- 29m 34s) (5000 33%) 5.9158
17m 40s (- 26m 31s) (6000 40%) 5.8018
20m 38s (- 23m 35s) (7000 46%) 5.8331
23m 34s (- 20m 37s) (8000 53%) 5.8553
26m 35s (- 17m 43s) (9000 60%) 5.8881
29m 36s (- 14m 48s) (10000 66%) 5.8838
32m 32s (- 11m 50s) (11000 73%) 5.7826
35m 32s (- 8m 53s) (12000 80%) 5.8217
38m 27s (- 5m 55s) (13000 86%) 5.7463
41m 28s (- 2m 57s) (14000 93%) 5.9020
44m 22s (- 0m 0s) (15000 100%) 5.8471


In [ ]:
def BLEU_score(encoder, decoder, n = len(test_pairs)):
  score = 0
  for i in range(n):
        pair = random.choice(test_pairs)
        reference = [pair[1].split(' ')]
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        candidate = output_sentence.split(' ')
        i_score = sentence_bleu(reference, candidate)
        score = score + i_score
  avg_score = score/n

  return(avg_score)

In [ ]:
base_score = BLEU_score(encoder1,attn_decoder1)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
tuned_score = BLEU_score(trained_encoder,trained_decoder)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
print("Number of Training Pairs : ", len(train_pairs))
print("Number of Validation Pairs : ", len(val_pairs))
print("Number of Test Pairs : ", len(test_pairs))
print("Base Model Score : ", base_score*100)
print("Tuned Model Score : ", tuned_score*100)

Number of Training Pairs :  21340
Number of Validation Pairs :  6097
Number of Test Pairs :  3049
Base Model Score :  22.937959302089144
Tuned Model Score :  23.389528609976853
